# Importing Libraries:

In [1]:
from re import A
import nltk
import numpy as np
from nltk.stem.porter import PorterStemmer
import torch
import torch.nn as nn
import json
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import random
nltk.download('punkt')

In [6]:
# Load the data

with open('intents.json', 'r') as f:
    intents = json.load(f)

# Data Preprocessing: (Defining Functions)

In [4]:
stemmer = PorterStemmer()

def tokenize(sentence):
    return nltk.word_tokenize(sentence)

def stem(word):
    return stemmer.stem(word.lower())

def bag_of_words(tokenized_sentence, all_words):

    tokenized_sentence = [stem(w) for w in tokenized_sentence]

    bag = np.zeros(len(all_words), dtype=np.float32)
    for idx, w in enumerate(all_words):
        if w in tokenized_sentence:
            bag[idx] = 1.0

    return bag

# Neural Network: (Creating the Model)

In [5]:
# creating the neural network that we'll use to train the data
# We define the layers in the first function
# In the second function, we are passing the inputs through the NN. 
# Every output is passed through the ReLU function before going to the next layer 

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size) 
        self.l2 = nn.Linear(hidden_size, hidden_size) 
        self.l3 = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        # no activation and no softmax at the end
        return out

# Applying Data Preprocessing functions:

In [7]:
all_words = []
tags = []
xy = []
for intent in intents['intents']:
    tag = intent['tag']
    tags.append(tag)
    for pattern in intent['patterns']:
        w = tokenize(pattern)  # tokenizing all words in the patterns (potential questions)
        all_words.extend(w)  # we use extend instead of append because pattern is already an array and we don't want to put an array in an array
        xy.append((w, tag))
    
ignore_words = ['?', '!', '.', ',']
all_words = [stem(w) for w in all_words if w not in ignore_words] #stemming all the words and ignoring punctuation 
all_words = sorted(set(all_words))  # using set() removes duplicate elements
tags = sorted(set(tags))  #adds unique labels (not necessary but do it in to be safe) 

# Creating Training Data:

In [8]:
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    bag = bag_of_words(pattern_sentence, all_words) 
    X_train.append(bag)

    label = tags.index(tag)
    y_train.append(label) #cross entropy loss
    
X_train = np.array(X_train)
y_train = np.array(y_train)

In [9]:
# This step is not necessary, but it makes any potential debugging a lot easier.

class ChatDataset(Dataset):
    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    #dataset[idx]
    def __getitem__(self,index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.n_samples

# Training Process:

In [10]:
# Training Parameters

batch_size = 8
hidden_size = 8
output_size = len(tags)
input_size = len(X_train[0])
learning_rate = 0.001
num_epochs = 1000

In [11]:
# initializing the dataset and model

dataset = ChatDataset()
train_loader = DataLoader(dataset = dataset, batch_size = batch_size, shuffle = True, num_workers = 0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = NeuralNet(input_size, hidden_size, output_size).to(device)
    
#Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [12]:
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)

        # forward
        outputs = model(words)
        loss = criterion(outputs, labels)

        #backward and optimizer step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if (epoch +1) % 100 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

#print(f'final loss, loss={loss.item():.4f}')

Epoch [100/1000], Loss: 0.9485
Epoch [200/1000], Loss: 0.1328
Epoch [300/1000], Loss: 0.0527
Epoch [400/1000], Loss: 0.0090
Epoch [500/1000], Loss: 0.0080
Epoch [600/1000], Loss: 0.0025
Epoch [700/1000], Loss: 0.0047
Epoch [800/1000], Loss: 0.0009
Epoch [900/1000], Loss: 0.0007
Epoch [1000/1000], Loss: 0.0008


In [14]:
print(f'final loss, loss={loss.item():.4f}')

final loss, loss=0.0008


# Save Model and Data:

In [15]:
data = {
    "model_state" : model.state_dict(),
    "input_size" : input_size,
    "output_size" : output_size,
    "hidden_size" : hidden_size,
    "all_words" : all_words,
    "tags" : tags
}

FILE = "data.pth"
torch.save(data,FILE)

# Load model:

In [16]:
FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

# Let's chat ;)

In [18]:
model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state) # loading the learnt parametres
model.eval()

bot_name = "Sam"
print("Let's chat! (type 'quit' to exit)")
while True:
    sentence = input("You: ")
    if sentence == "quit":
        break

    sentence = tokenize(sentence)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0]) # converting to just one row since we have a single sentence
    X = torch.from_numpy(X).to(device) # converting to torch tensor

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    # during training, crossentropy loss automatically applies softmax.
    # to get maximum probability as the prediction; we manually apply the softmax function below
    
    
    probs = torch.softmax(output, dim=1) 
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                print(f"{bot_name}: {random.choice(intent['responses'])}")
    else:
        print(f"{bot_name}: I do not understand...")

Let's chat! (type 'quit' to exit)
You: hi
Sam: Hi there, what can I do for you?
You: one joke please
Sam: I do not understand...
You: do you know some joke
Sam: Why did the hipster burn his mouth? He drank the coffee before it was cool.
You: hahaha
Sam: I do not understand...
You: okay bye then
Sam: Bye! Come back again soon.
You: quit


# Resources:

https://studentsxstudents.com/creating-a-chatbot-with-pytorch-495622d77481


https://github.com/SurjaHead/Chatbot-with-PyTorch


https://www.python-engineer.com/posts/chatbot-pytorch/
